##### Title: Gasoline Demand Prediction using RandomForestRegressor and GradientBoostingRegressor
##### Date: 2024
##### Author: Kingstone Sithole

##### Introduction
This project aims to predict the demand for four different gasoline products (petrol, diesel, das, and jet fuel) using RandomForestRegressor and GradientBoostingRegressor. We will analyze the impact of variables such as Month, Year, and GDP on the monthly demand for these products. By understanding the relationship between these variables and the demand, we can develop predictive models to forecast future demand, which can help in supply chain management and strategic planning.

##### Data Description

The dataset contains monthly demand data for four gasoline products (petrol, diesel, das, and jet fuel) from January 2009 to December 2023. The other file GDP.xls, contains yearly GDP for various countries from 1960 to 2022

Loading required Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from numpy.random import seed
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import warnings as wr
wr.filterwarnings('ignore')

#### Data Loading and Preprocessing

In [13]:
# Read data
# Here we are reading data from excel
gdp_data = pd.read_excel(
    "GDP.xls", sheet_name="Data")
diesel_demand = pd.read_excel(
    "Petrolem Demand.xlsx", sheet_name="Diesel_Finalized")
petrol_demand = pd.read_excel(
    "Petrolem Demand.xlsx", sheet_name="Petrol_Finalized")
gas_demand = pd.read_excel("Petrolem Demand.xlsx", sheet_name="LPG_Finalized")
parafin_demand = pd.read_excel("Petrolem Demand.xlsx", sheet_name="Paraffin_Finalized")
jet_demand = pd.read_excel("Petrolem Demand.xlsx", sheet_name="Jet A1_ Finalized")


In [ ]:
#View top 10 of the records
diesel_demand.head()

In [ ]:
gdp_data.head()

In [29]:
#Reindex and drops unwanted rows and re index
diesel_demand.columns = diesel_demand.iloc[0]
diesel_demand.drop(index=0, axis=0, inplace=True)

gas_demand.columns = gas_demand.iloc[0]
gas_demand.drop(index=0, axis=0, inplace=True)

petrol_demand.columns = petrol_demand.iloc[0]
petrol_demand.drop(index=0, axis=0, inplace=True)

parafin_demand.columns = parafin_demand.iloc[0]
parafin_demand.drop(index=0, axis=0, inplace=True)

jet_demand.columns =jet_demand.iloc[0]
jet_demand.drop(index=0, axis=0, inplace=True)

In [14]:
gdp_data.columns = gdp_data.iloc[2]
gdp_data.drop(index=[0, 1, 2], axis=0, inplace=True)

<h4>Data cleaning and preprocessing steps</h4>
<mark>
There are 6 dataset, gdp and 5 others for the different petroleum Products. We would like to join them together such that it will be easy to analyse. 
</mark>

In [98]:

#Select row which contains ZWE only
zwe_gdp = gdp_data[gdp_data['Country Code'] == "ZWE"]

#Drop year that are not wanted
zwe_gdp = zwe_gdp.drop(columns=['Country Name',   'Country Code', 'Indicator Name', 'Indicator Code',
                                1960.0,           1961.0,           1962.0,           1963.0,
                                1964.0,           1965.0,           1966.0,           1967.0,
                                1968.0,           1969.0,           1970.0,           1971.0,
                                1972.0,           1973.0,           1974.0,           1975.0,
                                1976.0,           1977.0,           1978.0,           1979.0,
                                1980.0,           1981.0,           1982.0,           1983.0,
                                1984.0,           1985.0,           1986.0,           1987.0,
                                1988.0,           1989.0,           1990.0,           1991.0,
                                1992.0,           1993.0,           1994.0,           1995.0,
                                1996.0,           1997.0,           1998.0,           1999.0,
                                2000.0,           2001.0,           2002.0,           2003.0,
                                2004.0,           2005.0,           2006.0,           2007.0,
                                2008.0], axis=1)
zwe_gdp.dropna(inplace=True, axis=1)
zwe_gdp[2023] = np.mean(np.array(zwe_gdp))
# Step 1: Reset the index of the Series
zwe_gdp=zwe_gdp.reset_index().drop(columns="index")
zwe_gdp=pd.DataFrame({"Year":pd.to_numeric(zwe_gdp.columns).astype(int),"GDP":zwe_gdp.loc[0].values})

In [100]:
#Convert Columns Into Rows for all petroleum fuel datasets
month_columns = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
                 'August', 'September', 'October', 'November', 'December']

# Melt the month columns into rows
diesel_df = diesel_demand.melt(id_vars=['Year'], value_vars=month_columns, var_name='Month', value_name='Diesel')
parafin_df =parafin_demand.melt(id_vars=['Year'], value_vars=month_columns, var_name='Month', value_name='Parafin')
petrol_df = petrol_demand.melt(id_vars=['Year'], value_vars=month_columns, var_name='Month', value_name='Petrol')
gas_df = gas_demand.melt(id_vars=['Year'], value_vars=month_columns, var_name='Month', value_name='Gas')
jet_df=jet_demand.melt(id_vars=['Year'], value_vars=month_columns, var_name='Month', value_name='Jet')

In [115]:
df=diesel_df.merge(zwe_gdp,on="Year",how="outer")
df=petrol_df.merge(df,on=["Year", "Month"],how="outer")
df=gas_df.merge(df,on=["Year", "Month"],how="outer")
df=jet_df.merge(df,on=["Year", "Month"],how="outer")

In [116]:
df.head()

,Year,Month,Jet,Gas,Petrol,Diesel,GDP
0,2015,January,2828852,1084265,36955185,58041555,1.996312e+10
1,2016,January,2119361,1749203,39633390,55688710,2.054868e+10
2,2017,January,5081075,2586660,31253611,51240381,1.758489e+10
3,2018,January,5019247,2856100,51440816,62094755,3.415607e+10
4,2019,January,5085708,2627800,49092634,69296899,2.183223e+10


Merge Fuel dataset with GDP

In [ ]:
#Since GDP is for whole year, we want for as specfic month only, so will use average(divide by 12)
data_df['GDP'] = data_df['GDP']/12

#Convert Date col into date time
data_df['Date'] = pd.to_datetime(data_df['Date'])

#Sort the Data by 'Date'
data_df.sort_values(by="Date",inplace=True)

#Split Date into years and months
data_df['Year'] = data_df['Date'].dt.year
data_df['Month'] = data_df['Date'].dt.month


In [ ]:
#Save the transformed Data into excel
data_df.to_excel("Transformed Data.xlsx",index=False)

Data Exploration and Analysis

In [ ]:
# Summary statistics
print(data_df.describe())

In [ ]:
#Plottinf line graph for Demand Vs Date 
plt.figure(figsize=(14, 5))
sns.lineplot(data_df,y='Demand',x="Date")
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter("%Y"))
plt.title('Line Plot, Demand vs Date')
plt.show()

<mark>From 2009 to 2014, Demand was increasing gradually but then it falls in 2015, since then Demand was stationary untill 2023. SInce this is for all the products, there might be one product which was not performing well from 2015.</mark> We would like to analyse the graph for each and every product

In [ ]:
droducts = data_df["Product"].unique()
sns.set_style("darkgrid")
# Plot distribution of each numerical feature
for idx, product in enumerate(droducts, 1):
    plt.figure(figsize=(14, 5))
    sns.lineplot(data_df[data_df["Product"] == product], y="Demand", x="Date")
    plt.title(f"{product} Demand")
    plt.xlabel("Year")
    plt.show()

<mark>From the above Graphs</mark>
<ul>
<li>Gas Demand was increasing from 2009 </li>
<li>Jet Demand was increasing but fell to a lowest point between 2020 and 2021. Then it increase in high momentum</li>
<li>The demand of Parafin  fell and continues to fall from 2017</li>
<li>Petrol and Diesel are flactuating</li>
</ul>

Analyse relationship between GDP VS Demand

In [ ]:
plt.figure(figsize=(14, 5))
sns.lineplot(data_df,y='Demand',x="GDP")
plt.title('Line Plot, GDP vs Demand')
plt.show()

Correlation Between  GDP and Demand

In [ ]:
df_grouped_by_year=data_df[data_df['Date'].dt.year >= 2015]
df_grouped_by_year['Year'] = df_grouped_by_year['Date'].dt.year
df_grouped_by_year=df_grouped_by_year.groupby(["Year"])

# Assuming 'df' is your DataFrame
plt.figure(figsize=(10,3))

# Using Seaborn to create a heatmap
sns.heatmap(df_grouped_by_year[['GDP', 'Demand']].sum().corr(), annot=True, fmt='.2f', cmap='Pastel2', linewidths=2)

plt.title('Correlation Heatmap')
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))
# Sales vs Customers Scatter Plot
sns.scatterplot(x=df_grouped_by_year[['GDP', 'Demand']].sum()["GDP"], y=df_grouped_by_year[['GDP', 'Demand']].sum()["Demand"])
plt.title("GDP vs Demand")
plt.show()

Model Development and Evaluation

<p>We come up with two Models, RandomForestRegressor and GradientBoostingRegressor. The problem will be Modelled as a regression problem than a time series problem. Note that we have an option to treat Diesel,Petrol,Jet,Parafin and Gas as separate variables however we have treated them as Categorial in one variable Product.
</p>
<p>
In our approach one can predict Demand by providing year,month and the product the want to predict.
</p>

In [ ]:
#Create a new Dataframe from the original
df_new=data_df.copy()

# Convert 'Product' to categorical codes
df_new['Product_codes'] = df_new['Product'].astype('category').cat.codes


Select X and Y

In [ ]:
features=['Month', 'Year', 'GDP','Product_codes']
target=["Demand"]
X = df_new[features]  # Independent variables
Y = df_new[target]  # Independent variables
Y = np.array(Y)
X = np.array(X)

Scaling the data

In [ ]:
#Scale data into values between 0 and 1
x_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))
x_scaled=x_scaler.fit_transform(X)
y_scaled=y_scaler.fit_transform(np.array(Y.reshape(-1,1)))


# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    x_scaled, y_scaled, test_size=0.2)


y_test_actual=pd.DataFrame(y_scaler.inverse_transform(y_test.reshape(-1,1)),columns=['Demand'])
y_test=y_test.ravel()

Random Forest-Model

In [ ]:
#Train RandomForest Model
random_forest_model = RandomForestRegressor(
    n_estimators=200, criterion="poisson")
random_forest_model.fit(x_train,y_train.ravel())

# Make predictions on the testing set
random_forest_predict = random_forest_model.predict(x_test)


# Evaluate the model's performance
rf_score = r2_score(y_test, random_forest_predict)
rf_mse= mean_squared_error(y_test, random_forest_predict)
rf_mae =mean_absolute_error(y_test, random_forest_predict)
rf_rmse= mean_squared_error(y_test, random_forest_predict)

print("================Measurement Metrics================================")
print(f"R^2 Squared={rf_score}")
print(f"MSE={rf_mse}")
print(f"MAE={rf_mae}")
print(f"RMSE={rf_rmse}")


Visualise The results for Random Forest

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(y_test, label='Actual', color='red')
plt.plot(random_forest_predict, label='Prediction', color='blue')

plt.xlabel('Index')
plt.ylabel('Demand')
plt.legend()
plt.title('Random Forest')
plt.show()

<mark>
The graph above shows two graphs for Predicted Demand vs The Actual. These are values which include all the product and without respect to time, but indices.
<br>
We can see that Random Forest Regression model,predicted well in most of the values, however, it was not able to predict well in outliere
</mark>

GradientBoostingRegressor

In [ ]:
gradient_boosting_model = GradientBoostingRegressor(
    n_estimators=207, loss="squared_error", alpha=0.01)
gradient_boosting_model.fit(x_train, y_train.ravel())
gradient_boosting_predict = gradient_boosting_model.predict(x_test)

print("================Measurement Metrics (Gradient Boosting)================================")
gb_score = r2_score(y_test, gradient_boosting_predict)
gb_mse= mean_squared_error(y_test, gradient_boosting_predict)
gb_mae =mean_absolute_error(y_test, gradient_boosting_predict)
gb_rmse= mean_squared_error(y_test, gradient_boosting_predict)

print(f"R^2 Squared={gb_score}")
print(f"MSE={gb_mse}")
print(f"MAE={gb_mae}")
print(f"RMSE={gb_rmse}")

Visualise results for Gradient Boosting

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(y_test, label='Actual', color='red')
plt.plot(gradient_boosting_predict, label='Prediction', color='blue')

plt.xlabel('Index')
plt.ylabel('Demand')
plt.legend()
plt.title('Gradient Boosting Regression Actual vs Predicted')
plt.show()

In [ ]:
gb_pred_actual=pd.DataFrame(y_scaler.inverse_transform(gradient_boosting_predict.reshape(-1,1)),columns=['Demand'])
gb_pred_actual=gb_pred_actual.map('{:.0f}'.format)
gb_pred_actual = gb_pred_actual.apply(pd.to_numeric)

Using the Best Model To make prediction for product

In [ ]:
best_model=gradient_boosting_model if gb_mse>rf_mse  else random_forest_model

Predict For Diesel

In [ ]:
def predict_(product: str):
    product_df = df_new[df_new['Product'] == product]
    product_x = np.array(product_df[features])
    product_predicted = best_model.predict(x_scaler.transform(product_x))
    product_predicted = y_scaler.inverse_transform(
        product_predicted.reshape(-1, 1))
    product_predicted = pd.DataFrame(
        product_predicted, columns=['Demand_predicted'])
    product_predicted.index = product_df.index
    product_df = pd.concat([product_df, product_predicted], axis=1)
    return product_df

##### Visualising the Prediction vs Actual for each product

In [ ]:
for product in df_new.Product.unique():
    plt.figure(figsize=(14,4))
    product_df=predict_(product)
    sns.lineplot(data=product_df,y="Demand",x="Date",label="Actaul")
    sns.lineplot(data=product_df,y="Demand_predicted",x="Date",label="Predicted")
    plt.xlabel('Date')
    plt.ylabel('Demand')
    plt.legend()
    plt.title(f'Actual vs Predicted [{product}]')
    plt.show()

##Conclusiong

<p>Based on the evaluation metrics, we can conclude that the model demonstrates an excellent fit and performs exceptionally well in predicting the Demand.</p>
<p>The R-squared value of 0.9347 indicates that the model explains 93.47% of the variability in the response variable, implying a strong correlation between the predicted and actual values. The high R-squared value suggests that the chosen features are relevant and contribute significantly to the prediction.</p>
<p>The MSE (Mean Squared Error) and RMSE (Root Mean Squared Error) are both quite low, with values of 0.00341 and 0.00340996, respectively. These metrics quantify the model's prediction error, and the small values indicate that the model's predictions closely match the actual observed values.</p>
<p>Furthermore, the MAE (Mean Absolute Error) of 0.03468 indicates a small average magnitude of the errors in the model's predictions. This further demonstrates that the model is effective in capturing the underlying patterns in the data and producing reliable estimates.</p>
<p>From the visualisation of the predicted vs Actual, we have noticed that the model tried to fit well in Diesel and Petrol, however it was unable to predict well in outliers.</p>

<p>In summary, the model exhibits strong performance in predicting the target variable, as demonstrated by its high R-squared value, low MSE, RMSE, and MAE. This suggests that the model is effective and reliable for its intended purpose. However, it is essential to continuously monitor and evaluate the model's performance over time to ensure its predictive power remains stable and robust.</p>
